<img src="monai.png" style="width: 700px;"/>

Welcome to the MONAI bootcamp! This tutorial demos the process of packaging a trained model using MONAI Deploy App SDK into an artifact that can be run as a local application that performs inference, a workflow job doing the same, and a Docker containerized workflow execution.

### Using Google Colab

This notebook has the pip command for installing MONAI and will be added to any subsequent notebook.

**Required Packages for Colab Execution**

Execute the following cell to install MONAI the first time a colab notebook is run:

In [ ]:
!python -c "import pydicom" || pip install -qU "pydicom"
!python -c "import monai" || pip install -qU "monai[nibabel]"
!python -c "import monai.deploy" || pip install -qU "monai-deploy-app-sdk"

**Note:** If you're having issues with imports, try to "Restart Kernel" for Jupyter from the "Kernel" dropdown menu.

**Enabling GPU Support**

To use GPU resources through Colab, change the runtime to GPU:

1. From the **"Runtime"** menu select **"Change Runtime Type"**
2. Choose **"GPU"** from the drop-down menu
3. Click **"SAVE"**

This will reset the notebook and probably ask you if you are a robot (these instructions assume you are not). Running

**!nvidia-smi**

in a cell will verify this has worked and show you what kind of hardware you have access to.    

In [ ]:
!nvidia-smi

# Deploying a MedNIST Classifier App with MONAI Deploy App SDK (Prebuilt Model)

This tutorial demos the process of packaging a trained model using MONAI Deploy App SDK into an artifact that can be run as a local application that performs inference, a workflow job doing the same, and a Docker containerized workflow execution.

### Contents
* [Setup](#setup)
* [Creating Operator Class](#create_op)
* [Creating Application Class](#create_app)
* [Executing App Locally](#execute_app_local)
* [Write Files to Disk](#write_files)
* [Run App using CLI](#run_cli)
* [Packing App as Docker Image](#package_app)
* [Exercise](#exercise)
* [Conclusion](#conclusion)

## Implementing and Packaging Application with MONAI Deploy App SDK

Based on the Torchscript model(`classifier.zip`), we will implement an application that processes an input Jpeg image and write the prediction(classification) result as a JSON file(`output.json`).

In our inference application, we will define two operators:

1. `LoadPILOperator` - Load a JPEG image from the input path and pass the loaded image object to the next operator.
    - This Operator does similar job with `LoadImage(image_only=True)` transform in *train_transforms*, but handles only one image.
    - **Input**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
2. `MedNISTClassifierOperator` - Pre-transform the given image by using MONAI's `Compose` class, feed to the Torchscript model (`classifier.zip`), and write the prediction into a JSON file(`output.json`)
    - Pre-transforms consist of three transforms -- `AddChannel`, `ScaleIntensity`, and `EnsureType`.
    - **Input**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output**: a folder path that the prediction result(`output.json`) would be written ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))

The workflow of the application would look like this.

<img src="images/mednist_workflow.png" alt="Workflow" style="width: 600px;margin-left:auto;margin-right:auto;"/>


<a id='setup'></a>
## 0 Setup

### 0.1 Setup environment

Let's import necessary classes/decorators and define `MEDNIST_CLASSES`. We'll also set up a variable to point to the 02_files folder, where we're storing data throughout this notebook.

In [ ]:
NOTEBOOK_ROOT="data/02_files/"

**Note:** If you're having issues with imports, try to "Restart Kernel" for Jupyter from the "Kernel" dropdown menu.

#### Download Data required for all of the MONAI Deploy Notebooks
We're going to download sample images and python files required for running the MONAI Deploy Jupyter Notebooks.

In [ ]:
!wget https://github.com/zephyrie/monai-bootcamp/releases/download/monai-deploy-data-v0.1/deploy_data.zip
!unzip deploy_data.zip

### 0.2 Setup imports

Let's import necessary classes/decorators and define `MEDNIST_CLASSES`.

In [ ]:
import monai.deploy.core as md
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]

from monai.deploy.operators.dicom_text_sr_writer_operator import DICOMTextSRWriterOperator, EquipmentInfo, ModelInfo
from typing import Text

<a id='create_op'></a>
## 1. Creating Operator classes

### 1.1 LoadPIL Operator

In [ ]:
@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """Load image from the given input (DataPath) and set numpy array to the output (Image)."""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        image = PILImage.open(input_path)
        image = image.convert("L")  # convert to greyscale image
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # create Image domain object with a numpy array
        op_output.set(output_image)

### 1.2 MedNIST Classifier Operator

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.output("result_text", Text, IOType.IN_MEMORY)
@md.env(pip_packages=["monai"])
class MedNISTClassifierOperator(Operator):
    """Classifies the given image and returns the class name."""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # get a TorchScriptModel object

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MEDNIST_CLASSES[output_classes[0]]  # get the class name
        print(result)
        op_output.set(result, "result_text")

        ##### Get output (folder) path and create the folder if not exists
        # output_folder = op_output.get().path
        # output_folder.mkdir(parents=True, exist_ok=True)

        ### The following gets the App context's output path, instead the operator's.
        output_folder = context.output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)
        
        
        # Write result to "output.json"
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)

<a id='create_app'></a>
## 2. Creating Application class

Our application class would look like below.

It defines `App` class inheriting `Application` class.

`LoadPILOperator` is connected to `MedNISTClassifierOperator` by using `self.add_flow()` in `compose()` method of `App`.

In [ ]:
@md.resource(cpu=1, gpu=1, memory="1Gi")
class App(Application):
    """Application class for the MedNIST classifier."""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MedNISTClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)
        
        # Hint for the exercise 
        
#         my_model_info = ModelInfo("MONAI WG Trainer", "MEDNIST Classifier", "0.1", "xyz")
#         my_equipment = EquipmentInfo(manufacturer="MONAI Deploy App SDK", manufacturer_model="DICOM SR Writer")
#         my_special_tags = {"SeriesDescription": "Not for clinical use. The result is for research use only."}
#         dicom_sr_operator = DICOMTextSRWriterOperator(
#             copy_tags=False, model_info=my_model_info, equipment_info=my_equipment, custom_tags=my_special_tags
#         )
        
#         self.add_flow(classifier_op, dicom_sr_operator, {"result_text": "classification_result"})

<a id='execute_app_local'></a>
## 3. Executing app locally

We can execute the app in the Jupyter notebook.

In [ ]:
app = App()
app.run(input=NOTEBOOK_ROOT+"/input/AbdomenCT_007000.jpeg", output=NOTEBOOK_ROOT+"/output", model=NOTEBOOK_ROOT+"classifier.zip")

In [ ]:
!cat $NOTEBOOK_ROOT/output/output.json

<a id='write_file'></a>
## 4. Write code to Disk

Once the application is verified inside the Jupyter notebook, we can write the whole application as a file(`mednist_classifier_monaideploy.py`) by concatenating the code above, then add the following lines:

```python
if __name__ == "__main__":
    App(do_run=True)
```

The above lines are needed to execute the application code by using `python` interpreter.

We have already done this for you, and you can open the `02_files/mednist_classifier_monaideploy.py` file by double-clicking on it from the left sidebar of JupyterLab. 

We're utilizing an existing application provided by MONAI Deploy App SDK.  You can find it in the MONAI Deploy App SDK GitHub Repo in the `examples/app/mednist_classifier_monaideploy` folder: https://github.com/Project-MONAI/monai-deploy-app-sdk/tree/main/examples/apps/mednist_classifier_monaideploy.



<a id='run_cli'></a>
## 5. Run App on CLI
You can run the App in one of two ways:
- Using the python command
- Using the 'monai-deploy' command 

---
 > **Note**: We are executing python code which makes developing and debugging simple.
---

In [ ]:
!python {NOTEBOOK_ROOT}"/mednist_classifier_monaideploy.py" -i {NOTEBOOK_ROOT}"/input/AbdomenCT_007000.jpeg" -o {NOTEBOOK_ROOT}"/output" -m {NOTEBOOK_ROOT}"/classifier.zip"

The above command and below command both run the application, but one uses python directly, and the other uses the provided MONAI Deploy CLI.

In [ ]:
!monai-deploy exec {NOTEBOOK_ROOT}"/mednist_classifier_monaideploy.py" -i {NOTEBOOK_ROOT}"/input/AbdomenCT_007000.jpeg" -o {NOTEBOOK_ROOT}"/output" -m {NOTEBOOK_ROOT}"/classifier.zip"

We'll then use the `cat` comand to display the output result.

In [ ]:
!cat {NOTEBOOK_ROOT}/output/output.json

<a id='package_app'></a>
## 6. Package app (creating MAP Docker image)

Let's package the app with [MONAI Application Packager](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/packaging_app.html).

Below we'll use the monai-deploy CLI to package the application by using the package command.  We'll pass the `simple_imaging_app` folder path and use the `simple_app` tag. We'll use the `-l DEBUG` option to see the progress.

In [ ]:
#!monai-deploy package {NOTEBOOK_ROOT}"/mednist_classifier_monaideploy.py" \
#    --tag mednist_app:latest \
#    --model {NOTEBOOK_ROOT}"/classifier.zip" -l DEBUG

After creating the docker image, we can verify it by using the `docker image ls` command and grepping for `mednist_app`.

In [ ]:
#!docker image ls | grep mednist_app

### 6.1 Executing packaged app locally

The packaged app can be run locally through [MONAI Application Runner](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/executing_packaged_app_locally.html).

In [ ]:
#!monai-deploy run mednist_app:latest "${NOTEBOOK_ROOT}/input" "${NOTEBOOK_ROOT}/dockerOutput"

### 6.2 Check the output from the packaged app
Now we can see the results from the packaged docker app.

In [ ]:
#!cat {NOTEBOOK_ROOT}/output.json

<a id='exercise'></a>
## 7. Exercise
### 7.1 Generate DICOM Structured report Dicom SR  

Most findings need to be structured in a report and displayed to the clinicians in the form of dicomSR. MONAI Deploy already supports DicomSR, and you can simply add the operator to the end of your app.

Use lines below to add on to the application
``` 
my_model_info = ModelInfo("MONAI WG Trainer", "MEDNIST Classifier", "0.1", "xyz")
my_equipment = EquipmentInfo(manufacturer="MONAI Deploy App SDK", manufacturer_model="DICOM SR Writer")
my_special_tags = {"SeriesDescription": "Not for clinical use. The result is for research use only."}
dicom_sr_operator = DICOMTextSRWriterOperator(
    copy_tags=False, model_info=my_model_info, equipment_info=my_equipment, custom_tags=my_special_tags
)

self.add_flow(classifier_op, dicom_sr_operator, {"result_text": "classification_result"})
```

### 7.2 Check the DICOM Structured Report output

You can render the DICOM SR in most DICOM viewers. For now, let's inspect the DICOM tags and see the text in the body.

In [ ]:
import pydicom

In [ ]:
!ls -la {NOTEBOOK_ROOT}/output

**Note:** Add one of the DICOM files from `ls` output in previous cell to as the dcmSRPath.

In [ ]:
#replace dcmSRPath with file produced 
dcmSRPath="1.2.826.0.1.3680043.8.498.12345993046274368708659346300123756138_SR.dcm"
pydicom.read_file(NOTEBOOK_ROOT+"output/"+dcmSRPath) 

<a id='conclusion'></a>
## Conclusion

In this notebook, we have walked through the process to create a classification task using a pre-built model using MONAI Deploy App SDK.  You've run the application in Jupyter, locally using Python and the MONAI Deploy CLI, and finally you packaged the application using docker and executed the newly created container image.

### What's Next
The following notebook we'll walk through a more in-depth organ segmentation task.